## Imports

In [107]:
# adds parent directory to python path so we can access code located there
import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path: sys.path.append(nb_dir)
    
# core imports
from ohmeow_ml.keras_tf_util import *

# other imports
from IPython.display import FileLink

# configure matplotlib
%matplotlib inline
    
# configure autoreload to re-load changed modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define paths and global variables

In [108]:
current_dir = os.getcwd()
DATA_HOME_DIR = current_dir + '/data/'
DATA_CLASSES = [ dir for dir in os.listdir(DATA_HOME_DIR+'train') ]

# path = DATA_HOME_DIR
path = DATA_HOME_DIR + 'sample/'
sample_path = DATA_HOME_DIR + 'sample/'

train_path = path + 'train/'
val_path = path + 'valid/'
test_path = path + 'test/'

models_path = path + 'models/'                      # save weights here
results_path = path + 'results/'                    # save predictions here
processed_data_path = path + 'preprocesed_data/'    # save preprocessed data used for training here

if not os.path.exists(models_path): os.makedirs(models_path)
if not os.path.exists(results_path): os.makedirs(results_path)
if not os.path.exists(processed_data_path): os.makedirs(processed_data_path)

In [109]:
batch_size = 4 #64

## Submission

In [ ]:
def do_clip(arr, mx):
    return np.clip(arr, (1-mx)/9, mx)

def create_submission(preds, filename='subm.gz'):
    subm = do_clip(preds, 0.93)
    subm_file = results_path+filename
    
    batches = get_batches(train_path, batch_size=1, shuffle=False)
    classes = sorted(batches.class_indices, key=batches.class_indices.get)
    
    df_subm = pd.DataFrame(subm, columns=classes)
    df_subm.insert(0, 'img', [a[8:] for a in test_filenames])
    #print(df_subm.head())
    
    df_subm.to_csv(subm_file, index=False, compression='gzip')
    return subm_file

## Preprocess the data

We can save time by pre-processing the images (e.g., converting them to jpegs, resizing to 224x224) and saving them as a numpy array on the file system.  We can do the same for the train, validation, and test image class designations, filenames, and one-hot encoded labels

In [110]:
# get classes, one-hot encoded labels, and filenames
train_classes, train_labels, train_filenames = get_batch_info(train_path)
val_classes, val_labels, val_filenames = get_batch_info(val_path)
test_filenames = get_batch_info(test_path)[2]

Found 1500 images belonging to 10 classes.
Found 750 images belonging to 10 classes.
Found 500 images belonging to 1 classes.


In [111]:
 # get image data
if not os.path.exists(processed_data_path+'train_data.bc'):
    train_data = get_data(train_path)
    save_array(processed_data_path+'train_data.bc', train_data)
else:
    train_data = load_array(processed_data_path+'train_data.bc')
    print('training data loaded ...')

if not os.path.exists(processed_data_path+'val_data.bc'):
    val_data = get_data(val_path)
    save_array(processed_data_path+'val_data.bc', val_data)
else:
    val_data = load_array(processed_data_path+'val_data.bc')
    print('validation data loaded ...')

# NOTE: with almost 80k records, trying to serialize the test set results in a memory error
# if not os.path.exists(processed_data_path+'test_data.bc'):
#     test_data = get_data(test_path)
#     save_array(processed_data_path+'test_data.bc', test_data)
# else:
#     test_data = load_array(processed_data_path+'test_data.bc')
#     print('test data loaded ...')

training data loaded ...
validation data loaded ...


Create training/validation batches and also define "steps per epoch" for each ... defines the # of batches per epoch (see `model.fit_generator()`).

***ONLY RUN THIS CODE IF YOU NEED TO USE BATCHES INSTEAD OF PERSISTED IMAGE ARRAYS***

In [6]:
# OPTION 1: BUILD BATCHES FROM FILE SYSTEM
# train_batches = get_batches(train_path, batch_size=batch_size)
# val_batches = get_batches(val_path, batch_size=batch_size*2, shuffle=False)

# OPTION 2: BUILD BATCHES FROM IMAGE ARRAYS
# gen = image.ImageDataGenerator()
# train_batches = gen.flow(train_data, train_labels, batch_size=batch_size, shuffle=True)
# val_batches = gen.flow(val_data, val_labels, batch_size=batch_size*2, shuffle=False)

# DEFINE # OF STEPS TO TAKE IN FITTING BATCHES FOR BOTH TRAINING AND VALIDATION EXAMPLES
# epoch_steps = math.ceil(train_batches.n/train_batches.batch_size)
# val_steps = math.ceil(val_batches.n/val_batches.batch_size)

## Simple CNN

2 conv layers with max pooling + a simple dense network is a good simple CNN to start with

In [13]:
def simple_cnn():
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(224,224,3)),
        Conv2D(32, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])

    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

### Without Data Augmentation

In [14]:
limit_mem()
model = simple_cnn()

In [15]:
model.fit(train_data, train_labels, batch_size=batch_size, epochs=2, shuffle=True, 
          validation_data=(val_data, val_labels), verbose=2)

Train on 16951 samples, validate on 5473 samples
Epoch 1/2
578s - loss: 0.6795 - acc: 0.8251 - val_loss: 1.9279 - val_acc: 0.3967
Epoch 2/2
573s - loss: 0.1113 - acc: 0.9874 - val_loss: 1.8846 - val_acc: 0.4550


In [16]:
model.optimizer.lr = 0.001
model.fit(train_data, train_labels, batch_size=batch_size, epochs=5, shuffle=True, 
          validation_data=(val_data, val_labels), verbose=2)

Train on 16951 samples, validate on 5473 samples
Epoch 1/5
574s - loss: 0.0513 - acc: 0.9965 - val_loss: 1.8822 - val_acc: 0.4679
Epoch 2/5
573s - loss: 0.0303 - acc: 0.9989 - val_loss: 1.8765 - val_acc: 0.4650
Epoch 3/5
574s - loss: 0.0196 - acc: 0.9998 - val_loss: 1.8850 - val_acc: 0.4688
Epoch 4/5
574s - loss: 0.0141 - acc: 0.9998 - val_loss: 1.8204 - val_acc: 0.4836
Epoch 5/5
573s - loss: 0.0105 - acc: 0.9999 - val_loss: 1.7986 - val_acc: 0.4931


In [17]:
model.save_weights(models_path+'simple_cnn_weights.h5') # val_acc = 0.4931

### With Data Augmentation

In [18]:
# get the best values
df_augs = pd.read_csv(sample_path+'data_augmentation_results.csv')
df_augs.sort_values('val_acc', ascending=False).groupby('aug').first()

,aug_val,train_loss,train_acc,val_loss,val_acc
aug,,,,,
channel_shift_range,10.00,0.464624,0.902444,1.735638,0.464444
height_shift_range,0.00,0.431795,0.917333,1.985304,0.436000
rotation_range,10.00,0.817478,0.770444,1.790981,0.476444
shear_range,0.00,0.431570,0.912667,1.761170,0.480444
width_shift_range,0.05,0.867643,0.755333,1.771413,0.515556
zoom_range,0.00,0.445235,0.899778,1.787687,0.477778


In [19]:
gen_aug = image.ImageDataGenerator(channel_shift_range=10.0, height_shift_range=0.0, rotation_range=10.0, 
                                   shear_range=0.0, width_shift_range=0.05, zoom_range=0.0)

aug_batches = gen_aug.flow(train_data, train_labels, batch_size=batch_size, shuffle=True)

In [20]:
limit_mem()
model = simple_cnn()

In [21]:
epoch_steps = math.ceil(aug_batches.n/aug_batches.batch_size)
model.fit_generator(aug_batches, epoch_steps, epochs=2, validation_data=(val_data, val_labels), verbose=2)

Epoch 1/2
577s - loss: 1.4254 - acc: 0.5438 - val_loss: 1.8220 - val_acc: 0.3768
Epoch 2/2
575s - loss: 0.5995 - acc: 0.8449 - val_loss: 1.7175 - val_acc: 0.4952


In [22]:
model.optimizer.lr = 0.001
model.fit_generator(aug_batches, epoch_steps, epochs=4, validation_data=(val_data, val_labels), verbose=2)

Epoch 1/4
575s - loss: 0.3676 - acc: 0.9169 - val_loss: 1.6963 - val_acc: 0.5085
Epoch 2/4
574s - loss: 0.2698 - acc: 0.9427 - val_loss: 1.6049 - val_acc: 0.5363
Epoch 3/4
574s - loss: 0.2061 - acc: 0.9589 - val_loss: 1.5723 - val_acc: 0.5461
Epoch 4/4
575s - loss: 0.1687 - acc: 0.9680 - val_loss: 1.5861 - val_acc: 0.5407


In [24]:
model.optimizer.lr = 0.0001
model.fit_generator(aug_batches, epoch_steps, epochs=4, validation_data=(val_data, val_labels), verbose=2)

Epoch 1/1
577s - loss: 0.0623 - acc: 0.9901 - val_loss: 1.5507 - val_acc: 0.5620


In [25]:
model.save_weights(models_path+'simple_cnn_da_weights.h5') # val_acc = 0.5620

## Complex CNN Architecutre

We are adding in regularization via Dropout so this will work better on full data set

In [30]:
def complex_cnn(p_do=0.5, n_dense_outputs=256):
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(224,224,3)),
        Conv2D(32, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Conv2D(128, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        
        Flatten(),
        Dense(n_dense_outputs, activation='relu'),
        BatchNormalization(),
        Dropout(p_do/4),
        Dense(n_dense_outputs, activation='relu'),
        BatchNormalization(),
        Dropout(p_do),
        Dense(10, activation='softmax')
    ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [31]:
limit_mem()
model = complex_cnn()

In [32]:
gen_aug = image.ImageDataGenerator(channel_shift_range=10.0, height_shift_range=0.0, rotation_range=10.0, 
                                   shear_range=0.0, width_shift_range=0.05, zoom_range=0.0)

aug_batches = gen_aug.flow(train_data, train_labels, batch_size=batch_size, shuffle=True)
epoch_steps = math.ceil(aug_batches.n/aug_batches.batch_size)

In [33]:
model.fit_generator(aug_batches, epoch_steps, epochs=2, validation_data=(val_data, val_labels), verbose=2)

Epoch 1/2
636s - loss: 1.7451 - acc: 0.4699 - val_loss: 2.1765 - val_acc: 0.3170
Epoch 2/2
634s - loss: 0.6807 - acc: 0.7763 - val_loss: 1.7729 - val_acc: 0.5081


In [34]:
model.optimizer.lr = 0.001
model.fit_generator(aug_batches, epoch_steps, epochs=4, validation_data=(val_data, val_labels), verbose=2)

Epoch 1/4
633s - loss: 0.4139 - acc: 0.8669 - val_loss: 1.6830 - val_acc: 0.5695
Epoch 2/4
634s - loss: 0.2573 - acc: 0.9206 - val_loss: 1.4854 - val_acc: 0.6141
Epoch 3/4
633s - loss: 0.1832 - acc: 0.9451 - val_loss: 1.4465 - val_acc: 0.6386
Epoch 4/4
634s - loss: 0.1550 - acc: 0.9540 - val_loss: 1.6164 - val_acc: 0.5752


In [36]:
model.optimizer.lr = 0.0001
model.fit_generator(aug_batches, epoch_steps, epochs=6, validation_data=(val_data, val_labels), verbose=2)

Epoch 1/4
633s - loss: 0.0887 - acc: 0.9746 - val_loss: 1.6451 - val_acc: 0.6464
Epoch 2/4
632s - loss: 0.0756 - acc: 0.9782 - val_loss: 1.5715 - val_acc: 0.6521
Epoch 3/4
632s - loss: 0.0657 - acc: 0.9810 - val_loss: 1.7755 - val_acc: 0.6168
Epoch 4/4
633s - loss: 0.0628 - acc: 0.9809 - val_loss: 1.7813 - val_acc: 0.6366


In [38]:
model.optimizer.lr = 0.00001
model.fit_generator(aug_batches, epoch_steps, epochs=8, validation_data=(val_data, val_labels), verbose=2)

Epoch 1/4
633s - loss: 0.0380 - acc: 0.9887 - val_loss: 1.7406 - val_acc: 0.5978
Epoch 2/4
633s - loss: 0.0387 - acc: 0.9887 - val_loss: 1.5417 - val_acc: 0.6592
Epoch 3/4
633s - loss: 0.0309 - acc: 0.9906 - val_loss: 1.8036 - val_acc: 0.6338
Epoch 4/4
631s - loss: 0.0340 - acc: 0.9906 - val_loss: 1.8142 - val_acc: 0.6203


In [ ]:
model.save_weights(models_path+'complex_cnn_da_weights.h5') # val_acc = ~0.63

## Finetune Pre-Trained Models

### 1. Train a linear classifier using the pre-computed output from 2nd to last layer

In [57]:
limit_mem()
model = VGG19(weights='imagenet', include_top=True)

In [58]:
# pop last layer and set model.outputs = to that of the now last layer
model.layers.pop()

# model.layers[-1].outbound_nodes = [] ... this is not needed
model.outputs = [model.layers[-1].output]

In [60]:
# model.summary()

#### Pre-compute output for train, validation, test data

In [61]:
# A. precompute the 2nd to last layer for training and validation data sets
if not os.path.exists(processed_data_path+'train_features_ll.bc'):
    train_features_ll = model.predict(train_data, 4)
    val_features_ll = model.predict(val_data, 4)
    
    save_array(processed_data_path+'train_features_ll.bc', train_features_ll)
    save_array(processed_data_path+'val_features_ll.bc', val_features_ll)
else:
    train_features_ll = load_array(processed_data_path+'train_features_ll.bc')
    val_features_ll = load_array(processed_data_path+'val_features_ll.bc')
    
print('training data:', train_features_ll.shape)
print('validation data:', val_features_ll.shape)

# B. do the same for augmented training data ... make this 5-10x larger
if not os.path.exists(processed_data_path+'da_train_features_ll.bc'):
    da_gen = image.ImageDataGenerator(channel_shift_range=10.0, height_shift_range=0.0, rotation_range=10.0, 
                                   shear_range=0.0, width_shift_range=0.05, zoom_range=0.0)

    # shuffle=False because we are going to have to add labels later for however many 
    # augmented sets of the training data
    da_batches = get_batches(train_path, da_gen, batch_size=batch_size, shuffle=False) 
    da_train_features_ll = np.concatenate(
        [ model.predict_generator(da_batches, (da_batches.n/batch_size), verbose=2) for i in range(5)])
    
    save_array(processed_data_path+'da_train_features_ll.bc', da_train_features_ll)
else:
    da_train_features_ll = load_array(processed_data_path+'da_train_features_ll.bc')
    
print('augmented data:', da_train_features_ll.shape)

# C. do the same for test data
if not os.path.exists(processed_data_path+'test_features_ll.bc'):
    test_batches = get_batches(test_path, batch_size=batch_size, shuffle=False)
    test_features_ll = model.predict_generator(test_batches, (test_batches.n/batch_size), verbose=2)
    save_array(processed_data_path+'test_features_ll.bc', test_features_ll)
else:
    test_features_ll = load_array(processed_data_path+'test_features_ll.bc')
    
print('test data:', test_features_ll.shape)

training data: (1500, 4096)
validation data: (750, 4096)
augmented data: (7500, 4096)
test data: (500, 4096)


In [62]:
all_train_features_ll = np.concatenate([da_train_features_ll, train_features_ll])
all_train_labels_ll = np.concatenate([train_labels]*6)

print('all training features shape:', all_train_features_ll.shape)
print('all training labels shape:', all_train_labels_ll.shape)

all training features shape: (9000, 4096)
all training labels shape: (9000, 10)


#### Finetune

In [63]:
for layer in model.layers: layer.trainable = False
ft_ll_model = Sequential([ Dense(10, activation='softmax', input_shape=model.layers[-1].output_shape[1:]) ])

In [64]:
ft_ll_model.compile(optimizer=Adam(lr=1e-05), loss='categorical_crossentropy', metrics=['accuracy'])
ft_ll_model.fit(all_train_features_ll, all_train_labels_ll, batch_size=batch_size, epochs=5, 
          validation_data=(val_features_ll, val_labels), verbose=2)

Train on 9000 samples, validate on 750 samples
Epoch 1/5
8s - loss: 2.5488 - acc: 0.1506 - val_loss: 2.5022 - val_acc: 0.1587
Epoch 2/5
8s - loss: 2.1464 - acc: 0.2343 - val_loss: 2.3253 - val_acc: 0.1907
Epoch 3/5
8s - loss: 1.9491 - acc: 0.2921 - val_loss: 2.2579 - val_acc: 0.2320
Epoch 4/5
8s - loss: 1.8190 - acc: 0.3374 - val_loss: 2.1852 - val_acc: 0.2333
Epoch 5/5
8s - loss: 1.7207 - acc: 0.3658 - val_loss: 2.1545 - val_acc: 0.2627


In [65]:
ft_ll_model.optimizer.lr = 0.01
ft_ll_model.fit(all_train_features_ll, all_train_labels_ll, batch_size=batch_size, epochs=5, 
          validation_data=(val_features_ll, val_labels), verbose=2)

Train on 9000 samples, validate on 750 samples
Epoch 1/5
8s - loss: 1.6441 - acc: 0.3917 - val_loss: 2.1306 - val_acc: 0.2787
Epoch 2/5
8s - loss: 1.5805 - acc: 0.4150 - val_loss: 2.1393 - val_acc: 0.2773
Epoch 3/5
8s - loss: 1.5275 - acc: 0.4347 - val_loss: 2.0904 - val_acc: 0.2800
Epoch 4/5
8s - loss: 1.4796 - acc: 0.4434 - val_loss: 2.0754 - val_acc: 0.2813
Epoch 5/5
8s - loss: 1.4391 - acc: 0.4631 - val_loss: 2.1100 - val_acc: 0.2827


In [67]:
ft_ll_model.save_weights(models_path+'ft_ll_model_weights.h5') # val_acc = ???

OSError: Unable to create file (Unable to truncate a file which is already open)

### 2. Precompute the convolutional and use in FC NN

In [112]:
limit_mem()
model = VGG19(include_top=False, weights='imagenet', input_shape=(224,224,3)) # must include input_shape if include_top=False

In [113]:
# model.summary()

#### Precompute output for train, validation, test data

In [114]:
# A. precompute the 2nd to last layer for training and validation data sets
if not os.path.exists(processed_data_path+'train_features_conv.bc'):
    train_features_conv = model.predict(train_data, 4)
    val_features_conv = model.predict(val_data, 4)
    
    save_array(processed_data_path+'train_features_conv.bc', train_features_conv)
    save_array(processed_data_path+'val_features_conv.bc', val_features_conv)
else:
    train_features_conv = load_array(processed_data_path+'train_features_conv.bc')
    val_features_conv = load_array(processed_data_path+'val_features_conv.bc')
    
print('training data:', train_features_conv.shape)
print('validation data:', val_features_conv.shape)

# B. do the same for augmented training data ... make this 5-10x larger
if not os.path.exists(processed_data_path+'da_train_features_conv.bc'):
    da_gen = image.ImageDataGenerator(channel_shift_range=10.0, height_shift_range=0.0, rotation_range=10.0, 
                                   shear_range=0.0, width_shift_range=0.05, zoom_range=0.0)

    # shuffle=False because we are going to have to add labels later for however many 
    # augmented sets of the training data
    da_batches_conv = get_batches(train_path, da_gen, batch_size=batch_size, shuffle=False) 
    da_train_features_conv = np.concatenate(
        [ model.predict_generator(da_batches_conv, (da_batches_conv.n/batch_size), verbose=2) for i in range(5) ])
    
    save_array(processed_data_path+'da_train_features_conv.bc', da_train_features_conv)
else:
    da_train_features_conv = load_array(processed_data_path+'da_train_features_conv.bc')
    
print('augmented data:', da_train_features_conv.shape)

# C. do the same for test data
if not os.path.exists(processed_data_path+'test_features_conv.bc'):
    test_batches_conv = get_batches(test_path, batch_size=batch_size, shuffle=False)
    test_features_conv = model.predict_generator(test_batches_conv, (test_batches_conv.n/batch_size), verbose=2)
    save_array(processed_data_path+'test_features_conv.bc', test_features_conv)
else:
    test_features_conv = load_array(processed_data_path+'test_features_conv.bc')
    
print('test data:', test_features_conv.shape)

training data: (1500, 7, 7, 512)
validation data: (750, 7, 7, 512)
augmented data: (7500, 7, 7, 512)
test data: (500, 7, 7, 512)


In [115]:
all_train_features_conv = np.concatenate([da_train_features_conv, train_features_conv])
all_train_labels_conv = np.concatenate([train_labels]*6)

print('all training features shape:', all_train_features_conv.shape)
print('all training labels shape:', all_train_labels_conv.shape)

all training features shape: (9000, 7, 7, 512)
all training labels shape: (9000, 10)


#### Finetune

In [116]:
def build_fc_layers(dropout_p=0.5, dense_output=256):
    return [
        Flatten(input_shape=model.layers[-1].output_shape[1:]),
        Dropout(dropout_p),
        Dense(dense_output, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_p),
        Dense(dense_output, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_p),
        Dense(10, activation='softmax')
    ]

In [119]:
fc_model = Sequential(build_fc_layers())
fc_model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [120]:
fc_model.fit(all_train_features_conv, all_train_labels_conv, batch_size=batch_size, epochs=1,
            validation_data=(val_features_conv, val_labels), verbose=2)

Train on 9000 samples, validate on 750 samples
Epoch 1/1
66s - loss: 2.3326 - acc: 0.2373 - val_loss: 1.5670 - val_acc: 0.3960


In [122]:
fc_model.optimizer.lr = 0.01
fc_model.fit(all_train_features_conv, all_train_labels_conv, batch_size=batch_size, epochs=4,
            validation_data=(val_features_conv, val_labels), verbose=2)

Train on 9000 samples, validate on 750 samples
Epoch 1/4
64s - loss: 1.3589 - acc: 0.4618 - val_loss: 1.3327 - val_acc: 0.5120
Epoch 2/4
65s - loss: 1.3067 - acc: 0.4729 - val_loss: 1.3173 - val_acc: 0.5160
Epoch 3/4


KeyboardInterrupt: 

In [123]:
fc_model.optimizer.lr = 0.001
fc_model.fit(all_train_features_conv, all_train_labels_conv, batch_size=batch_size, epochs=4,
            validation_data=(val_features_conv, val_labels), verbose=2)

Train on 9000 samples, validate on 750 samples
Epoch 1/4
64s - loss: 1.2683 - acc: 0.4803 - val_loss: 1.2923 - val_acc: 0.5880
Epoch 2/4
64s - loss: 1.2478 - acc: 0.4963 - val_loss: 1.4411 - val_acc: 0.5040
Epoch 3/4
64s - loss: 1.2132 - acc: 0.5012 - val_loss: 1.1812 - val_acc: 0.5933
Epoch 4/4
64s - loss: 1.1951 - acc: 0.5171 - val_loss: 1.3182 - val_acc: 0.5053


In [124]:
fc_model.optimizer.lr = 0.0001
fc_model.fit(all_train_features_conv, all_train_labels_conv, batch_size=batch_size, epochs=4,
            validation_data=(val_features_conv, val_labels), verbose=2)

Train on 9000 samples, validate on 750 samples
Epoch 1/4
64s - loss: 1.1709 - acc: 0.5247 - val_loss: 1.3111 - val_acc: 0.5000
Epoch 2/4
64s - loss: 1.1642 - acc: 0.5229 - val_loss: 1.3094 - val_acc: 0.4840
Epoch 3/4
65s - loss: 1.1435 - acc: 0.5334 - val_loss: 1.3587 - val_acc: 0.4840
Epoch 4/4
65s - loss: 1.1268 - acc: 0.5342 - val_loss: 1.3018 - val_acc: 0.5293


In [126]:
fc_model.save_weights(models_path+'ft_fc_model_weights.h5') # val_acc = ???

#### Evaluate and create submission

In [ ]:
fc_model.evaluate(val_features_conv, val_labels, batch_size=batch_size, verbose=2)

In [212]:
preds = fc_model.predict(test_features_conv, batch_size=batch_size, verbose=2)

In [213]:
subm_file = create_submission(preds, 'ft_fc_model_subm01.gz')
FileLink(subm_file)

Found 1500 images belonging to 10 classes.


C:\Development\_training\machine_learning\ml-fastai-tensorflow\kaggle_statefarm\data\sample\results\ft_fc_model_subm01.gz